# Xarray hands-on exercises

In [ ]:
import numpy as np
import xarray as xr

import pystac_client
import planetary_computer

import rioxarray
import xrspatial

import ipyleaflet
import matplotlib.pyplot as plt
import hvplot.xarray
from datashader.transfer_functions import shade, stack
from datashader.colors import Elevation

## First let's get some topographic data

In this example, we download SRTM elevation data from Microsoft's [Planetary Computer data catalog](https://planetarycomputer.microsoft.com/catalog) via the [STAC API](https://stacspec.org) (disclaimer: No API key is required here but this might change in the future, also STAC API and related tools are still pretty young).

More examples here: https://github.com/microsoft/PlanetaryComputerExamples

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

Let's create an interactive map ([ipyleaflet](https://ipyleaflet.readthedocs.io)) on which we can delimit a ROI:

In [ ]:
m = ipyleaflet.Map(
    basemap=ipyleaflet.basemaps.OpenTopoMap,
    center=[27.9881, 86.9250],
    zoom=11,
)

draw_control = ipyleaflet.DrawControl()
m.add_control(draw_control)
 
m

Search the data catalog (NASA DEM collection) for items that interesect the ROI:

In [ ]:
roi = draw_control.last_draw['geometry']

if roi is None:
    # take map center point as default
    roi = {'type': 'Point', 'coordinates': [m.center[1], m.center[0]]}

search = catalog.search(collections=["nasadem"], intersects=roi)
items = search.item_collection()

items

Each item has some assets that each provide a link to the actual data and metadata.

In [ ]:
if not items:
    raise ValueError("no item found, please select another ROI")

item = items[0]
item.assets

Now we can use [rioxarray](https://corteva.github.io/rioxarray/) to directly load the data (Cloud Optimized GeoTIFF, COG) as a `xarray.DataArray`:

In [ ]:
dem_raw = rioxarray.open_rasterio(item.assets["elevation"].href)

dem_raw

Let's clean-up and prepare the data, i.e.,

- remove the "band" dimension and coordinate (not relevant for elevation data)
- downsample the image a bit (the original image tile is quite large) with a 5x5 mean window
- reproject the data from lat/lon to UTM and convert no-data values

In [ ]:
utm_crs = dem_raw.rio.estimate_utm_crs()

dem = (
    dem_raw
    .squeeze()
    .drop("band")[:-1, :-1]
    .coarsen({"y": 5, "x": 5})
    .mean()
    .rio.reproject(utm_crs)
    .where(lambda da: da > 0.)
)

dem

Finally, let's make a quick plot of the data:

In [ ]:
dem.plot.imshow(yincrease=True);

Plotting elevation is nicer with some hillshading, here using [Xarray-spatial](https://xarray-spatial.org/) and [Datashader](https://datashader.org/).  

In [ ]:
shaded = xrspatial.hillshade(dem, azimuth=100, angle_altitude=50)
stack(
    shade(shaded, cmap=["white", "gray"]),
    shade(dem, cmap=Elevation, alpha=128, how="linear", span=[500, 6000])
)

## Exercise 1: cross-sections

Extract and plot one or several cross-sections along the `x` or `y` axis.

Hints:

- use Xarray's `sel()` or `isel()`
- see Xarray's [plotting guide](http://xarray.pydata.org/en/stable/user-guide/plotting.html)

In [ ]:
dem.y

In [ ]:
dem.sel(x=450e3, method='nearest').plot()

In [ ]:
dem.sel(y=3050e3, method='nearest').plot()

Create an interactive figure using `hvplot` where the position of the cross-section can be controlled with a slider

In [ ]:
dem.where(np.nan)

In [ ]:
dem.hvplot(groupby='x', frame_width=400, frame_height=400)

Extract a topographic profile given along a custom polyline defined from (`x`, `y`) points. Bonus: Draw a polyline on the map above and get it's coordinates.

Hints:

- Use Xarray's [advanced indexing](http://xarray.pydata.org/en/stable/user-guide/indexing.html#more-advanced-indexing) (pointwise selection)

In [ ]:
xp = np.linspace(dem.x.min() + 300, dem.x.max() - 300, 50) + np.random.uniform(-300, 300, 50) 
yp = np.linspace(dem.y.min() + 300, dem.y.max() - 300, 50) + np.random.uniform(-300, 300, 50)

In [ ]:
dem.sel(x=xr.DataArray(xp, dims='profile'), y=xr.DataArray(yp, dims='profile'), method='nearest').plot()
dem.interp(x=xr.DataArray(xp, dims='profile'), y=xr.DataArray(yp, dims='profile'), method='linear').plot();

In [ ]:
xr.concat(
    [
     dem.interp(x=xr.DataArray(xp, dims='profile'), y=xr.DataArray(yp, dims='profile'), method='linear'),
     dem.sel(x=xr.DataArray(xp, dims='profile'), y=xr.DataArray(yp, dims='profile'), method='nearest')
    ],
    dim='method'
).assign_coords({'method':['linear','nearest']}).plot.line(x='profile')

## Exercise 2: swath profiles

Extract and plot the mean/median/min/max elevation along the `x` or `y` axis. Bonus: gather all statistics into a single `xarray.DataArray` object and plot all the profiles with a legend using Xarray plotting methods.

Hints:

- use Xarray's [aggregation methods](http://xarray.pydata.org/en/stable/user-guide/computation.html#aggregation)
- See Xarray's [concatenate](http://xarray.pydata.org/en/stable/user-guide/combining.html#concatenate)

In [ ]:
fig, ax = plt.subplots()
plt.fill_between(dem.y, dem.min('x'), dem.max('x'), color='lightgrey')
(xr.concat([dem.mean('x'), dem.min('x'), dem.max('x'), dem.median('x')], dim='Topography')
 .assign_coords({'Topography':['mean','min','max','median']})
 .plot.line(ax=ax, x='y'));


## Exercise 3: Compute terrain derivatives

Compute and plot terrain slope using the following formula:

$$ s = \arctan \left( \sqrt{\frac{\partial{z}}{\partial{x}}^2 + \frac{\partial{z}}{\partial{y}}^2} \right) $$

For simplicity, let's ignore the diagnonal DEM grid neighbors in the computation of the partial derivatives. Convert the values in degrees.

Hints:

- Look at Xarray's [differentiate](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.differentiate.html#xarray.DataArray.differentiate)

In [ ]:
(np.arctan(np.sqrt((dem.differentiate('x'))**2+(dem.differentiate('y'))**2))/np.pi*180).plot()
plt.title('Slope (degrees)')

## Exercise 4: Load DEM mosaic as a single DataArray

Define a large ROI covering two or more STAC items, load the individual items and merge them as a single DataArray.

Hints:

- Use Xarray's [merge](https://docs.xarray.dev/en/stable/generated/xarray.merge.html)